In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read the csv files of problem logs
df = pd.read_csv("../input/learning-activity-public-dataset-by-junyi-academy/Log_Problem.csv")

In [ ]:
# Find out users that had reached level 4
df[df["level"]==4].head()

# Here are some examples

# uuid = z72ErFAjb2UfCFQ3plZd3yDe/D38BLs4tkS4sdD5azM=
# ucid = nds7oVk2a6yC/6tk1sTPm75v/0ZPkvrJ00WGmYRciOI=

# uuid = MxF194x6Pg2Ln1LI2rxQv2lTcICQFqjlE3gN4q6vets=
# ucid = BG5tyd78Q/W+eQU5e5tUTKS14+tiZ6//Zftq7fxiLYE=

# uuid = K3Wd3TlL2oqubVfU4g6oJvSk55TRoE0kGai0hVljdqc=
# ucid = 9sB9W+3x5hEFUN/pFLPakL+0ubNopXBy3NGjYgRPSI4=

### Please refer to the `What is level (Proficiency mechanism)?` section in the following notebook for more details.

https://www.kaggle.com/jason022085/an-exploratory-analysis-junyi-academy

### The summarized rules are as the following:

*  The level of each user for each content starts at 0.
*  The user has to complete enough problems under a certain content to raise the level.
*  Correctly answering 5 problems in a row of 6 problems will raise the level for the content from 0 to 1.
*  After level 1 until the exercise is Proficient (level 4)
   *  Continously correct in answering 2 problems  -> Upgrade.
   *  Continously wrong in answering 2 problems    -> Downgrade.
   *  One correct and one wrong attempt            -> No level change is made and we let the user try again.
*  There are time interval restrictions between upgrades as shown in the previous notebook for upgrades after reaching level 1.

The *is_downgrade* column will be False and *is_upgrade* column will be True whenever upgrade, and vice versa.

The *is_downgrade* and the *is_upgrade* column may be Null at the same time when the user has not answered enough problems to upgrade or downgrade or is still in the cooldown period for upgrading.

### Let's look at a example user

In [ ]:
# Let's look at a specific user and content
user_condition = (df["uuid"] == "z72ErFAjb2UfCFQ3plZd3yDe/D38BLs4tkS4sdD5azM=")
content_condition = (df["ucid"] == "nds7oVk2a6yC/6tk1sTPm75v/0ZPkvrJ00WGmYRciOI=")

# Sort by problem_number to follow user's progress
output = df[["problem_number","timestamp_TW","is_correct","is_downgrade","is_upgrade","level"]][user_condition & content_condition]
output = output.sort_values(by=["problem_number"]).reset_index(drop=True)

There are not enough correct attempts in the first 5 problem attempts.

When the user answers the 6th problem, the user have now correctly answered 5 problems in a row of 6 problems.

The level of the user becomes 1, which means False in *is_downgrade* and True in *is_upgrade*.

In [ ]:
output[17:23]

The user met some difficulties after reach level 1.

The user wrongly answer one problem and correctly answer the other as shown in the 23~24 row.

This results in no level change and we let the user try again.

After several attempts, the user finally continuosly answer 2 problems correct as shown in the 27~28 row.

The user level for this exercise now becomes 2, which means *is_downgrade* is False and *is_upgrade* is True.

In [ ]:
output[22:29]

On the next week, the user still meet some difficulties after reach level 2.

The user answered one problem correct and the other one wrong as shown in the 29~30 row.

Again, this results in no level change and we let the user try again.

The user continuously answer 2 problems wrong in 33~34 row.

The user level for this exercise now changes back to 1, which means *is_downgrade* is True and *is_upgrade* is False.

In [ ]:
output[29:35]